In [5]:
!jupyter nbconvert Game.ipynb --to script
!jupyter nbconvert Board.ipynb --to script
!jupyter nbconvert player/Player.ipynb --to script
!jupyter nbconvert player/Human.ipynb --to script
!jupyter nbconvert player/Randy.ipynb --to script
!jupyter nbconvert player/QLearner.ipynb --to script

[NbConvertApp] Converting notebook Game.ipynb to script
[NbConvertApp] Writing 3178 bytes to Game.py
[NbConvertApp] Converting notebook Board.ipynb to script
[NbConvertApp] Writing 8276 bytes to Board.py
[NbConvertApp] Converting notebook player/Player.ipynb to script
[NbConvertApp] Writing 990 bytes to player\Player.py
[NbConvertApp] Converting notebook player/Human.ipynb to script
[NbConvertApp] Writing 2010 bytes to player\Human.py
[NbConvertApp] Converting notebook player/Randy.ipynb to script
[NbConvertApp] Writing 667 bytes to player\Randy.py
[NbConvertApp] Converting notebook player/QLearner.ipynb to script
[NbConvertApp] Writing 2415 bytes to player\QLearner.py


In [1]:
import os
from Board import Result
from collections import defaultdict
from Decorators import debug, log, save_q_table
import numpy as np
import random
import dill
from abc import abstractmethod

class QLearner():



In [21]:
def get_q_table_from_file():
    q_table = defaultdict(lambda: np.zeros(9))

    if os.path.isfile('qtable.pkl') and os.path.getsize('qtable.pkl') > 0:
        with open('qtable.pkl', "rb") as qtablefile:
            q_table = dill.load(qtablefile)

    return q_table

In [78]:
#from player.QLearner import QLearner
from Board import Result
from collections import defaultdict
from player.Player import Player
from Decorators import debug, log
import numpy as np

class QPlayer(Player, QLearner):
    """
    Q learner player class
    """

    def __init__(self, representationChar, exp_factor=0.9, q_table_ttt=None):
        Player.__init__(self ,representationChar)
        self.actions = {0: [1,1],1:[1,2],2:[1,3],3:[2,1],4:[2,2],5:[2,3],6:[3,1],7:[3,2],8:[3,3]}
        QLearner.__init__(self, env_action_space=9, actions=self.actions, q_table=q_table_ttt)
        self.states_value = []  # record all positions taken
        self.exp_factor = exp_factor
        self.env_action_space=9
        self.q_table=defaultdict(lambda: np.zeros(self.env_action_space)) # dictionary
        self.prev_state = np.empty(shape=(3,3), dtype=Player)
        self.last_action = []
        self.action = []
        self.state = np.empty(shape=(3,3), dtype=Player)
        self.last_reward = 0

    @debug
    #@log(aiReadable=True)
    def makeMove(self, board):
        """
        Makes a move on the board
        :param board: current state of the board
        :return: list of a random column (first) and row number (second)
        """
        self.state = board.field
        self.action = QLearner.select_move(self,s=self.state)
        #print(self.last_action)
        return self.action

    def giveResult(self, result):
        """
        Set own reward for given result and updates the q table
        """
        reward = 0
        if Result.INVALID_MOVE == result: reward = -100
        if Result.GAME_LOST == result: reward = -10
        if Result.GAME_WON == result: reward = 100
        if Result.GAME_DRAW == result: reward = -1

        if len(self.last_action)>0:
            action_idx = list(self.actions.keys())[list(self.actions.values()).index(self.last_action)]
            qtable = QLearner.update(self, self.prev_state, self.state, action_idx, self.last_reward)
        #print(qtable)
        self.last_action = self.action
        self.prev_state = self.state
        self.last_reward = reward




In [79]:
from player.Randy import Randy
#from player.QLearnerBackup import QLearner
from Game import Game

In [80]:
def merge_defaultdicts(d,d1):
    for k,v in d1.items():
        if (k not in d):
            #d[k].update(d1[k])
            #d[k] = np.concatenate((d[k], d1[k]))
            d[k] = d1[k]
    return d

In [85]:
player1 = QPlayer('x', q_table_ttt=get_q_table_from_file())
player2 = QPlayer('o', q_table_ttt=get_q_table_from_file())
players = [player1,player2]

game = Game(players)
for i in range(1000000):
    game.run()

#print(player2.q_table)

dict = merge_defaultdicts(player1.q_table, player2.q_table)

with open('qtable.pkl', 'wb') as handle:
    dill.dump(dict, handle)

In [86]:
import dill
with open("qtable.pkl", "rb") as qtablefile:
    q_table = dill.load(qtablefile)

q_table


defaultdict(<function __main__.<lambda>()>,
            {"[[None None None]\n ['x' None 'o']\n [None None 'x']]": array([   0.        ,    0.        ,    0.        ,  -18.66001966,
                       0.        , -100.        ,    0.        ,    0.        ,
                      -3.54363533]),
             "[[None 'o' None]\n ['x' None 'o']\n [None None 'x']]": array([  0.        , -99.93734213,   0.        , -25.45817396,
                      0.        , -99.86899795,   0.        ,   0.        ,
                    -10.98521448]),
             "[['x' 'o' None]\n ['x' None 'o']\n [None None 'x']]": array([-29.67247605, -99.99982838,   0.        , -15.05904446,
                      0.        , -99.99970937,   0.        ,   0.        ,
                    -14.94595991]),
             "[['x' 'o' None]\n ['x' 'o' 'o']\n [None None 'x']]": array([-70.97163199, -99.99976459,   0.        , -29.8398383 ,
                    -99.99939236, -99.99885662,   0.        ,   0.        ,
         

In [77]:
import numpy as np
np.argmax(q_table["[['o' 'x' 'x']\n ['x' 'o' 'o']\n ['o' 'x' None]]"])

8

In [20]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:
import sys

from Board import Board
from Board import Result
from IPython.display import clear_output


# # Game class.
# This class is used to generate a new game to play.
# The game needs player to play it and a board to play on.
# This class also contains the logic of a game and a single turn.
# When the Game gets started with players, it will create a board and lets the first player make his move. When the move is incorrect, the game will send feedback to the player until he makes a valid move. The player gets feedback after a valid move. After the player makes a valid move, the game checks if the game is already over or not. When its over the players will get feedback and the game ends. When the game continous the next player can make his move.

# In[ ]:


class Game:

    def __init__(self, players=[]):
        """
        initializes the given players
        Parameters
        ----------
        players : list of player
                    can be a human player or an ai
        """
        self.players = players

    def turn(self,player,board):
        """
        Game logic for a player turn.
        Player makes a move and the board checks if the move is valid.
            When its valid the logic continues
            When its invalid the player gets feedback and has to do another move.

        Parameters
        ----------
         player : player
                 The current player
        board : board
                The board which needs the player inputs

        Return
        ------
        """
        pos = player.makeMove(board)
        is_valid = board.update(pos, player)
        while not is_valid:
            player.giveResult(Result.INVALID_MOVE)
            pos = player.makeMove(board)
            is_valid = board.update(pos, player)
        player.giveResult(Result.NO_RESULT)

    def run(self):
        """
        Logic of a complete Game.
        Initializes a new Board for every Game that is played.
        As long as the Game has no ending it continues.
            Every Player has his own turn.
            After a successful turn the game needs to check the state.
            When the Game is over and the player with his current turn wins, he will get notified with a victory.
                All other players ate getting the loose feedback.
            When the Game is over and its a draw everyone will get notified.
        Return
        ------
        """
        board = Board()
        state = Result.NO_RESULT
        while state == Result.NO_RESULT:
            for player in self.players:
                self.turn(player,board)
                state = board.over()
                if state == Result.GAME_WON:
                    for looser in self.players:
                        if not looser == player:
                            looser.giveResult(Result.GAME_LOST)
                    player.giveResult(state)
                    break
                elif state == Result.GAME_DRAW:
                    for playerDraw in self.players:
                        playerDraw.giveResult(state)
                    break



In [94]:
#!/usr/bin/env python
# coding: utf-8

# In[6]:
from IPython.display import clear_output


from player.Player import Player
from Board import Result
from Decorators import debug, log

class Human(Player):
    """
    Human player class for human participation in a game
    """

    @debug
    @log(humanReadable=True)
    def makeMove(self, board):
        """
        Prints a human readable game board starts input prompt
        :param board: current state of the board
        :return: list of selected column (first) and row (second)
        """
        #print(board)
        clear_output(wait=True)
        print(board)
        print("\n")
        action =  self.getColumn(), self.getRow()
        clear_output(wait=True)
        print(board)
        print("\n")
        return action

    @debug
    def getColumn(self):
        """
        Input prompt for column
        :return: selected column as integer
        """

        column = input("Enter column [A|B|C]: ")
        if column.__eq__("a") or column.__eq__("A"):
            return 1
        elif column.__eq__("b") or column.__eq__("B"):
            return 2
        elif column.__eq__("c") or column.__eq__("C"):
            return 3
        else:
            #self.giveResult(Result.WRONG_INPUT)
            column = self.getColumn()

        return column

    @debug
    def getRow(self):
        """
        Input prompt for row
        :return: selected row as integer
        """
        row = input("Enter row [1|2|3]: ")
        if (not (row.__eq__("1") or row.__eq__("2") or row.__eq__("3"))):
            #self.giveResult(Result.WRONG_INPUT)
            row = self.getRow()

        return int(row)

    @debug
    def giveResult(self, result):
        """
        Prints human readable feedback
        :param result: Enum element for feedback
        """

        if Result.INVALID_MOVE == result:
            print("This position is blocked or not on the board!")

        elif Result.GAME_LOST == result:
            print("You have lost the game. :/")

        elif Result.GAME_WON == result:
            print("You have won the game! :)")

        elif Result.GAME_DRAW == result:
            print("The game is over and no one won.")



In [95]:
player1 = QPlayer('x', get_q_table_from_file())
player2 = Human('o')
players = [player1,player2]

game = Game(players)
game.run()

o | x | o
x | x | None
o | None | x


You have lost the game. :/
